In [1]:
import optuna
from sklearn.cluster import KMeans
from keras.datasets import mnist
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

/workspaces/eargweth/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-17 23:24:51.148981: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-17 23:24:56.297252: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-17 23:24:58.411700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752794702.121192    1910 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752794703.026698    1910 cuda_blas.cc:1407] U

In [2]:
# Load and scale data
(x_train, y_train), (x_test, y_test) = mnist.load_data()



from numpy import concatenate


X = concatenate((x_train, x_test))

display(x_train.shape, x_test.shape)

X.shape

X = X.reshape(X.shape[0], -1)  # Flatten images

X.shape

X = X[:1000, :]

X.shape

(60000, 28, 28)

(10000, 28, 28)

(1000, 784)

In [25]:
def objective(trial):
    # Suggest hyperparameters
    n_clusters = trial.suggest_int('n_clusters', 2, 10)
    init_method = trial.suggest_categorical('init', ['k-means++', 'random'])
    max_iter = trial.suggest_int('max_iter', 100, 500)
    tol = trial.suggest_float('tol', 1e-5, 1e-1, log=True)

    # Create and fit KMeans
    kmeans = KMeans(
        n_clusters=n_clusters,
        init=init_method,
        max_iter=max_iter,
        tol=tol,
        random_state=42,
        n_init=10,
    )
    labels = kmeans.fit_predict(X)

    # Evaluate with silhouette score (higher is better)
    score = silhouette_score(X, labels)
    return score

In [26]:
# Create Optuna study to maximize silhouette score
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

best_trial = study.best_trial

best_params = best_trial.params

print(best_params)


[I 2025-07-15 22:15:58,588] A new study created in memory with name: no-name-67227f85-799a-47fd-ba33-ad5f15a3a646


[I 2025-07-15 22:15:58,838] Trial 0 finished with value: 0.07086093422393486 and parameters: {'n_clusters': 4, 'init': 'random', 'max_iter': 247, 'tol': 0.0550424470202509}. Best is trial 0 with value: 0.07086093422393486.
[I 2025-07-15 22:15:59,371] Trial 1 finished with value: 0.0776128439085069 and parameters: {'n_clusters': 5, 'init': 'k-means++', 'max_iter': 135, 'tol': 0.01059278227422711}. Best is trial 1 with value: 0.0776128439085069.
[I 2025-07-15 22:15:59,621] Trial 2 finished with value: 0.08752222881852827 and parameters: {'n_clusters': 2, 'init': 'random', 'max_iter': 278, 'tol': 0.0020164095405966233}. Best is trial 2 with value: 0.08752222881852827.
[I 2025-07-15 22:15:59,865] Trial 3 finished with value: 0.07834301567583539 and parameters: {'n_clusters': 6, 'init': 'random', 'max_iter': 416, 'tol': 0.00013280731248785472}. Best is trial 2 with value: 0.08752222881852827.
[I 2025-07-15 22:16:00,115] Trial 4 finished with value: 0.05826091260362078 and parameters: {'n_cl

{'n_clusters': 2, 'init': 'random', 'max_iter': 278, 'tol': 0.0020164095405966233}
